# Notebook for CAV2020 Artifact Evaluation - Submission 233

This notebook contains the tests to reproduce the results presented in our paper titled *Qualitative Controller Synthesis for Consumption Markov Decision Processes*. It also helps understand the package structure for ease of reusability.

Please note that the computation time for different tasks reported in the paper change based on the machine configuration but the scalabilty remains the same. The following computation times are obtained using a machine with the Intel® Core™ i7-8700 CPU @ 3.20GHz × 12 processor and a RAM of 16 GB running Ubuntu 18.04 LTS. This is a different configuration than the desktop used to generate the results reported in the paper and hence the computation times can be a little different. 

We provide detailed documentation and installation instructions using the documentation served using Sphinx and ReadtheDocs at the following links: [online version](https://fimdp.readthedocs.io/en/latest/) and [offline version - works in docker only](http://localhost:7777/view/docs/build/html/index.html).

While all the tests below are self-contained, we encourage you to have a look at other example notebooks. You can find brief descriptions of other example notebooks in the readme file in the current directory or at this [ReadtheDocs link](https://fimdp.readthedocs.io/en/latest/examples.html).

We consider the following two case-studies in our paper which model the problems of an electric vehicle routing with limited capacity and a multi-agent grid world inspired by the Mars 2020 mission. For detailed description of the problems, refer to the **Implementation and Case Studies** section of the paper or the following links:

* Description of Electric Vehicle Routing Problem: [ReadtheDocs link](https://fimdp.readthedocs.io/en/latest/examples.html#electric-vehicle-routing) or 
[Offline link - works in docker only](http://localhost:7777/view/docs/build/html/examples.html#electric-vehicle-routing)
* Description of Multi-agent Grid World Problem: [ReadtheDocs link](https://fimdp.readthedocs.io/en/latest/examples.html#multi-agent-grid-world) or 
[Offline link - works in docker only](http://localhost:7777/view/docs/build/html/examples.html#multi-agent-grid-world)


The following two subsections reproduce the results from the corresponding subsections in the **Implementation and Case Studies** section of the paper.

## Electric Vehicle Routing

This subsection reproduces the results in section *7.1 Electric Vehicle Routing* of the paper. We begin with importing required modules from fimdp.

In [10]:
import sys; sys.path.insert(0, '..')
import fimdp.nyc_parser as nyc_parser
import fimdp.NYCtools as NYCtools
from fimdp.energy_solver import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We generate the consMDP object representing the consumption MDP modeling the problem of electric vehicle routing in NYC. NYCstreetnetwork.json contains a stochastic energy consumption model covering all the streets in central Manhattan, New York City.

In [6]:
m, targets = nyc_parser.parse('NYCstreetnetwork.json')

### Mean computation time for different objectives

Mean computation time for *Safe* objective (safety) for an energy capacity of 100.

In [3]:
%%timeit
m.get_safe(100, recompute=True)

1.07 s ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafePR_T* (postive reachability) objective for an energy capacity of 100.

In [14]:
%%timeit
m.get_positiveReachability(targets, 100, recompute=True)

1.39 s ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafeBüchi_T* (Büchi) objective for an energy capacity of 100.

In [16]:
%%timeit
m.get_Buchi(targets, 100, recompute=True)

1.42 s ± 48.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Mean computation time for different capacities

We now analyze the variation of computation time for different capacities. The task is to generate a strategy with Buchi objective, same as the above experiments. We consider a maximum value of capacity 200 and split the interval [0, 200] into 20 intervals and use the interval bounds as the different values of capacities we calculate computation time for. Further, for each capacity value, we repeat the test multiple times to account for variations in the computation time. These results were reported in Fig. 3 of the paper. 

In [ ]:
comptime = NYCtools.timeit_diffcaps(m, targets, cap_bound = 200, num_samples = 20, num_tests=10, obj=BUCHI)
x_val = [pair[0] for pair in comptime]; y_val = [pair[1] for pair in comptime]
plt.scatter(x_val, y_val, color='red')
plt.xlabel('Capacity of the agent'); plt.ylabel('Computation Time (sec)'); plt.grid()
print(pd.DataFrame(comptime))

### Mean computation Time fo Different Sizes of Target Set

Now, we analyize how the computation time changes with the size of the targets. We randomly generate target sets of different sizes and calculate the computation time for the same capacity for Buchi objective. We use a capacity of 50 and generate target sets of varying sizes in the interval [20, size_bound] and use them to evaluate the strategy and calculate the computation time. For each target set, we repeat the tests multiple times to account for variations in the computation time.

In [ ]:
comptime = NYCtools.timeit_difftargetsizes(m, cap=50, size_bound = 200, num_samples = 20, num_tests=2, obj=BUCHI)
x_val = [pair[0] for pair in comptime]; y_val = [pair[1] for pair in comptime]
plt.scatter(x_val, y_val, color='red')
plt.xlabel('Size of the target set'); plt.ylabel('Computation Time (sec)'); plt.grid()

### Mean computation time for Different Target Sets

We now analyze how the computation time varies for different target sets. The function 'timeit_difftargets()' calculates the computation time of computing the strategy with Buchi objective for 'num_samples' different randomly generated target sets. The resulting compute time is plotted as a histogram as shown below. We consider a capacity of 100 and target set size of 50 for this analysis.

In [ ]:
comptime = NYCtools.timeit_difftargets(m, cap=100, target_size = 50, num_samples=100, obj=BUCHI)
NYCtools.histogram(comptime)

## Multi-agent Grid World

This subsection reproduces the results in section *7.2 Multi-agent Grid World* of the paper. We begin with importing required modules from fimdp. **Note that an environment with a grid size $n$ has $n^4$ states in the corresponding Consumption MDP.**

In [7]:
import sys; sys.path.insert(0, '..')
from fimdp.MarsEnv import MarsEnv
from fimdp.energy_solver import *

### Mean computation time for different objectives

We begin by creating a environment with grid size $n = 10$ and agent capacity of 10 units.

In [104]:
env = MarsEnv(grid_size=10, agent_capacity=10, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()

Mean computation time for *Safe* objective (safety) for an energy capacity of 10.

In [105]:
%%timeit
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(SAFE, recompute=True)

4.61 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafePR_T* (postive reachability) objective for an energy capacity of 10.

In [106]:
%%timeit
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(POS_REACH, recompute=True)

9.66 s ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafeBüchi_T* (Büchi) objective for an energy capacity of 10.

In [107]:
%%timeit
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(BUCHI, recompute=True)

9.85 s ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Mean computation time for Different Capacities and Grid Sizes

We calculate the computation time for obtaining the strategy for different sizes of grid (different sizes of state spaces) and different energy capacities for the same objective. These results were reported in Table 1 in the paper. We consider the Büchi objective and capacities of 10, 20, 50 and 100 for our tests. We note that the absolute values of the computation time vary on the machine used but should have the same trend in scalability.

In [98]:
def marsgrid_comptime(grid_sizes, cap):
    for grid_size in grid_sizes:
        env = MarsEnv(grid_size=grid_size, agent_capacity=cap, agent_actioncost=1, agent_staycost=1)
        m, targets = env.get_mdp_targets()
        t = %timeit -o -q s = EnergySolver(m, cap=env.agent_capacity, targets=targets); strategy = s.get_strategy(BUCHI, recompute=True)
        print('Computation time for grid size {} and capacity {}:'.format(grid_size, cap)); print(t)

#### Capacity = 10

**Grid sizes of $n = 5,7,10,15,20$, i.e., $625, 2401, 10000, 50625, 160000$ states.**

In [99]:
marsgrid_comptime([5,7,10,15,20],10)

Computation time for grid size 5 and capacity 10:
276 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 7 and capacity 10:
1.87 s ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 10 and capacity 10:
10.3 s ± 91.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 15 and capacity 10:
54.4 s ± 753 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 20 and capacity 10:
2min 59s ± 2.56 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Capacity = 20

**Grid sizes of $n = 5,7,10,15,20$, i.e., $625, 2401, 10000, 50625, 160000$ states.**

In [101]:
marsgrid_comptime([5,7,10,15,20],20)

Computation time for grid size 5 and capacity 20:
263 ms ± 5.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 7 and capacity 20:
1.9 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 10 and capacity 20:
12.3 s ± 33.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 15 and capacity 20:
1min 20s ± 231 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 20 and capacity 20:
4min 59s ± 706 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Capacity = 50

**Grid sizes of $n = 5,7,10,15,20$, i.e., $625, 2401, 10000, 50625, 160000$ states.**

In [102]:
marsgrid_comptime([5,7,10,15,20],50)

Computation time for grid size 5 and capacity 50:
282 ms ± 6.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 7 and capacity 50:
1.78 s ± 8.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 10 and capacity 50:
11.7 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 15 and capacity 50:
1min 30s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 20 and capacity 50:
6min 34s ± 1.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Capacity = 100

**Grid sizes of $n = 5,7,10,15,20$, i.e., $625, 2401, 10000, 50625, 160000$ states.**

In [103]:
marsgrid_comptime([5,7,10,15,20],100)

Computation time for grid size 5 and capacity 100:
263 ms ± 5.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 7 and capacity 100:
1.81 s ± 7.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 10 and capacity 100:
12.3 s ± 43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 15 and capacity 100:
1min 30s ± 308 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Computation time for grid size 20 and capacity 100:
6min 53s ± 1.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
